In [ ]:
import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, AutoConfig
import os
from datasets import load_from_disk
from vllm.inputs import TokensPrompt
from vllm import LLM, SamplingParams
from tqdm import tqdm
from small_llm_reasoning.evaluation.gsm8k import get_score, eight_shot_messages
# from small_llm_reasoning.generation.vllm_generation import llama_forward


cache_dir = '/scratch3/workspace/wenlongzhao_umass_edu-reason/dev_kedar/transformers_cache/'
os.environ['TRANSFORMERS_CACHE'] = cache_dir


/work/pi_mccallum_umass_edu/kchimmad_umass_edu/conda_envs/reason/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-12 19:43:31,613	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [ ]:
# Loading data
data_path= "../datasets/gsm8k"
split='train'
data = load_from_disk(f'{data_path}/{split}/')
data

In [ ]:
# Loading model
hf_token = os.getenv("hf_token")

# model_name= "meta-llama/Llama-3.2-1B-Instruct"
model_name= "meta-llama/Llama-3.2-3B-Instruct"
# model_name= "meta-llama/Llama-3.1-8B-Instruct"
# model_name= "meta-llama/Llama-3.2-3B"
# model_name= "meta-llama/Llama-3.3-70B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_token, cache_dir=cache_dir)
tokenizer.pad_token_id = tokenizer.eos_token_id



In [ ]:
def get_prompt(ex, few_shot):
    
    prompt = [
        {
            'role': 'user',
            'content': f'Given the following problem, reason and give a final answer to the problem.\nProblem: {ex}\nYour response should end with "The final answer is [answer]" where [answer] is the response to the problem.\n'
        }
    ]
    if few_shot:
        prompt = eight_shot_messages + prompt
    return prompt

def tokenize_function(example,few_shot):
    prompt= get_prompt(example['question'], few_shot)
    prompt= tokenizer.apply_chat_template(prompt,  tokenize= False, add_generation_prompt=True)
    return {'input_ids': {'prompt_token_ids':tokenizer(prompt, add_special_tokens=False)['input_ids']}}


In [ ]:
prompt=get_prompt(data['question'][0],True)
print(tokenizer.apply_chat_template(prompt,  tokenize= False, add_generation_prompt=True))
# print(prompt)

In [ ]:
# tokenized_dataset = data.map(tokenize_function, batched=False)
tokenized_dataset = data.map(lambda x: tokenize_function(x,few_shot=False), batched=False)


In [ ]:
len(tokenized_dataset['input_ids'])

In [ ]:
tokenized_dataset.save_to_disk(f"{data_path}/tokenized/LLaMA8B-Instruct/{split}/zero-shot/")